In [2]:
import pandas as pd
import os
from google import genai
import datetime
import time
from google.genai import types
from dotenv import load_dotenv
load_dotenv()
from response_types import *

GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
client = genai.Client(api_key=GEMINI_API_KEY)

In [3]:
model_name = "gemini-1.5-flash-002"

In [10]:
SYSTEM_PROMPT = """
You are to review a provided facility agreement in context to determine how the agreement affects the inclusion of assets in Significant Risk Transfers (SRTs).
Your purpose is to identify provisions regarding confidentiality, transferability, consents, confidentiality undertakings, approved lender lists, and any conditions that must be satisfied for including the asset in an SRT.
Your final output must be sufficiently clear, detailed, and legally accurate so that a legal team can rely on it directly.

In cases where the options for you to respond with are explicitly provided in the user prompt, you are required to respond with the most appropriate option and nothing else in your response.

Here is the full text of the facility agreement:
"""

In [5]:
questionnaire = pd.read_csv('/Users/vigneshramesh/Desktop/Deals/Natwest/question_tree.csv')
questionnaire.head(2)

,Question ID,Question Category,Question Text,Parent Question ID,Parent Answer Trigger,Answer Type,Options,Is Multi-select,Some GT Available,Questions to Legal Team
0,1,Agreement Details,Governing law,NaN,NaN,NaN,Not Specified|England and Wales|Scotland|North...,No,Yes,NaN
1,2,Agreement Details,Please detail the clause number of the governi...,NaN,NaN,Free Text,NaN,No,Yes,NaN


In [8]:
import PyPDF2

with open('/Users/vigneshramesh/Desktop/Deals/Natwest/SRT Checklist Groundtruths/Redacted_Facility_Agreement_12.pdf', 'rb') as f:
    pdf = PyPDF2.PdfReader(f)
    num_pages = len(pdf.pages)
    text = ''
    for page in range(num_pages):
        text += pdf.pages[page].extract_text()

In [11]:
contents = []

contents.append({
    "role": "system",
    "parts": [
        {
            "text": SYSTEM_PROMPT + "\n\n" + text
        }
    ]
})

In [ ]:

for category, group_df in questionnaire.groupby('Question Category'):
    print(f"Processing category: {category}")
    
    for _, row in group_df.iterrows():
        # Create base content object for each question
        content = {
            "role": "user",
            "parts": [
                {
                    "text": row['Question Text']
                }
            ]
        }
        
        # Check if question has a parent
        if pd.notna(row['Parent Question ID']):
            parent_id = int(row['Parent Question ID'])
            parent_trigger = row['Parent Answer Trigger']
            
            # Get parent question's response
            
                
        # Get response for current question
        response_schema = # Get the correct response schema based on the question from response_types.py
        response = client.models.generate_content(
            model=model_name,
            contents=contents,
            config=types.GenerateContentConfig(
                response_mime_type="application/json",
                response_schema=response_schema,
            ),
        )
        response_text = response.text
        df.at[index, 'Response'] = eval(response_text)
        
        # Add response as next content object
        contents.append(
            {
            "role": "assistant",
            "parts": [{"text": response.text}]
        }
        )

In [5]:
file_path = '/Users/vigneshramesh/Desktop/Deals/Natwest/SRT Checklist Groundtruths/Redacted_Facility_Agreement_12.pdf'
agreement_12 = client.files.upload(file=file_path, config=dict(mime_type='application/pdf'))

#Models offering cachine
1. gemini-1.5-pro-001
2. gemini-1.5-pro-002
3. gemini-1.5-flash-001
4. gemini-1.5-flash-002
5. gemini-1.5-flash-8b
6. gemini-1.5-flash-8b-001
7. gemini-1.5-flash-8b-latest

In [43]:
# Create a cached content object
cache = client.caches.create(
    model=model_name,
    config=types.CreateCachedContentConfig(
      system_instruction=SYSTEM_PROMPT,
      contents=[agreement_12]
    )
)

In [44]:
cache.name, cache.usage_metadata

('cachedContents/6jxi33kv6bro',
 CachedContentUsageMetadata(audio_duration_seconds=None, image_count=None, text_count=None, total_token_count=29543, video_duration_seconds=None))

In [45]:
contents = [
    {
    "role": "user",
    "parts": [
        {
            "text": "Is sub-participation defined?"
        }
    ],

    "role": "assistant",
    "parts": [
        {
            "text": """'{\n  "response": "Yes",\n  "rationale": "Clause 23.2(b) defines \\"sub-participation\\" to include any sub-participation, trust, or other contractual arrangement, assignment, or disposal entered into with any person or entity in respect of a Lender\'s rights and obligations under this Agreement (or any other transaction relating to the risks and returns of a Lender under this Agreement)."\n}'"""
        }
    ],

    "role": "user",
    "parts": [
        {
            "text": "Does the definition of sub-participation capture SRT?"
        }
    ],
    }
]


In [46]:
from google.genai.types import GenerateContentConfig

response = client.models.generate_content(
    model=model_name,
    contents='Does the definition of sub-participation capture SRT?',
    config=GenerateContentConfig(
        cached_content=cache.name,
        response_mime_type="application/json",
        response_schema=Agreement,
    ),
)

In [47]:
response.candidates[0].content.parts[0].text

'{\n  "response": "Yes",\n  "rationale": "The contract defines sub-participation broadly to include any sub-participation, trust or other contractual arrangement, assignment or disposal entered into with any person or entity in respect of a Lender\'s rights and obligations under this Agreement (or any other transaction relating to the risks and returns of a Lender under this Agreement)."\n}'

In [25]:
eval(response.text)

{'response': 'Yes',
 'rationale': 'Clause 23.2(b) defines "sub-participation" to include any sub-participation, trust, or other contractual arrangement, assignment, or disposal entered into with any person or entity in respect of a Lender\'s rights and obligations under this Agreement (or any other transaction relating to the risks and returns of a Lender under this Agreement).'}

In [30]:
for f in client.files.list():
    print("  ", f.name)

   files/sm2zudq3crut


In [29]:
client.files.delete(name='vfthbv1lgdyy')

DeleteFileResponse()

# SDK Methods

In [ ]:
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="Explain how AI works",
)

print(response.text)

In [ ]:
from google import genai
from google.genai import types
import pathlib
import httpx

client = genai.Client()

long_context_pdf_path = "https://www.nasa.gov/wp-content/uploads/static/history/alsj/a17/A17_FlightPlan.pdf" # Replace with the actual URL of your large PDF

# Retrieve the PDF
file_path = pathlib.Path('A17.pdf')
file_path.write_bytes(httpx.get(long_context_pdf_path).content)

# Upload the PDF using the File API
sample_file = client.files.upload(
  file=file_path,
)

prompt="Summarize this document"

response = client.models.generate_content(
  model="gemini-1.5-flash",
  contents=[sample_file, "Summarize this document"])
print(response.text)

In [9]:
from google import genai

chat = client.chats.create(model="gemini-2.0-flash")

response = chat.send_message("I have 2 dogs in my house.")
print(response.text)

response = chat.send_message("How many paws are in my house?")
print(response.text)

for message in chat.get_history():
    print(f'role - {message.role}',end=": ")
    print(message.parts[0].text)

Okay! That's great. Do you have any questions about having two dogs, or is there anything else I can help you with regarding your canine companions?

If you have two dogs and yourself, that means there are 8 dog paws (2 dogs x 4 paws each) and 2 human feet. Since paws are only on the dogs, there are **8 paws** in your house.

role - user: I have 2 dogs in my house.
role - model: Okay! That's great. Do you have any questions about having two dogs, or is there anything else I can help you with regarding your canine companions?

role - user: How many paws are in my house?
role - model: If you have two dogs and yourself, that means there are 8 dog paws (2 dogs x 4 paws each) and 2 human feet. Since paws are only on the dogs, there are **8 paws** in your house.



In [14]:
from google import genai

prompt = "Explain the concept of Occam's Razor and provide a simple, everyday example."
response = client.models.generate_content(
    model="gemini-2.5-pro-exp-03-25",  # or gemini-2.0-flash-thinking-exp
    contents=prompt
)

print(response.text)

Okay, let's break down Occam's Razor.

**The Concept of Occam's Razor**

Occam's Razor is a **problem-solving principle** or **heuristic** (a mental shortcut or guideline). It's not a strict law, but a suggestion for how to approach competing explanations for the same phenomenon.

In simple terms, it states:

**"When presented with competing hypotheses (explanations) for the same prediction, one should select the solution with the fewest assumptions."**

Often, this is paraphrased as:

**"The simplest explanation is usually the best one."**

Here's what that really means:

1.  **Competing Explanations:** It applies when you have *multiple possible ways* to explain something you observe.
2.  **Same Outcome:** All the explanations must actually account for the observed facts. If an explanation doesn't fit the facts, it's simply wrong, and Occam's Razor doesn't apply.
3.  **Fewest Assumptions:** The "simplest" explanation is the one that requires introducing the fewest new assumptions, va